# Modeling

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd

# load dataframe
df = pd.read_feather('../data/processed/twitter_tweets_processed.feather')
df.head(4)

,url,date,rawContent,preprocessed_text
0,https://twitter.com/YueDongCS/status/164159107...,2023-03-30 23:59:46,My condolences and sad that the #NLP and #AI c...,"[condolence, sad, nlp, ai, community, lose, br..."
1,https://twitter.com/Mlearning_ai/status/164159...,2023-03-30 23:59:43,Hiring Now: The Top Jobs of the Future Fueled ...,"[hiring, top, job, future, fueled, generative,..."
2,https://twitter.com/HackerAran7/status/1641591...,2023-03-30 23:59:43,What’s the hack. #stem #science #stemeducation...,"[hack, stem, science, stemeducation, education..."
3,https://twitter.com/Stemble_/status/1641590942...,2023-03-30 23:59:14,"🚀 Mark your calendars, Apple enthusiasts! 🗓️\n...","[mark, calendar, apple, enthusiast, june, 5, a..."


#### 1. Erstellen eines Wörterbuchs

Ein Wörterbuch ist wichtig für das LDA-Modell, da es das Modell trainiert, die Sprache und den Kontext der Dokumente zu verstehen. Es enthält alle eindeutigen Wörter, die in den Dokumenten vorkommen, und ordnet jedem Wort eine eindeutige Nummer zu. Das Wörterbuch wird verwendet, um jedes Dokument in eine Vektordarstellung umzuwandeln, die das Modell verwenden kann.

In [3]:
from gensim import corpora

dictionary = corpora.Dictionary(df['preprocessed_text'])

C:\Users\lukas\.virtualenvs\bachelorthesis-social-media-analytics-D4jMvGtF\Lib\site-packages\paramiko\transport.py:169: CryptographyDeprecationWarning: Blowfish has been deprecated
  'class': algorithms.Blowfish,


#### 2. Erstellen eines Corpus

Ein Corpus ist notwendig, um jedes Dokument in eine Vektordarstellung umzuwandeln, die vom LDA-Modell verarbeitet werden kann. Ein Corpus ist eine Sammlung von Dokumenten, die in eine Matrix umgewandelt wird, wobei jede Zeile für ein Dokument und jede Spalte für ein Wort im Wörterbuch steht. Die Matrix enthält die Anzahl der Vorkommen jedes Wortes in jedem Dokument. Durch die Umwandlung jedes Dokuments in eine Vektordarstellung kann das LDA-Modell jedes Dokument analysieren und Themen identifizieren, die in jedem Dokument vorkommen. Ohne diese Vektordarstellung könnte das Modell die Dokumente nicht analysieren und Themen identifizieren.

In [4]:
corpus = [dictionary.doc2bow(text) for text in df['preprocessed_text']]

#### 3. Erstellen eines LDA-Modells

In [5]:
from gensim import models

lda_model = models.LdaModel(corpus=corpus, id2word=dictionary, num_topics=10, passes=2)

In [6]:
from src import utils

_ = {
    'model': lda_model,
    'corpus': corpus,
    'dictionary': dictionary
}
utils.safe_as_pkl(_, 'lda_model', '../models')

#### 4. Ergebnisse visualisieren

In [1]:
from src import utils
import pyLDAvis.gensim_models

lda_model = utils.load_pkl('../models/lda_model.pkl')[0]

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model['model'], lda_model['corpus'], lda_model['dictionary'])
vis

C:\Users\lukas\.virtualenvs\bachelorthesis-social-media-analytics-D4jMvGtF\Lib\site-packages\paramiko\transport.py:169: CryptographyDeprecationWarning: Blowfish has been deprecated
  'class': algorithms.Blowfish,


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
8     -0.242046  0.062590       1        1  24.288596
4     -0.216893  0.116804       2        1  18.839607
2     -0.169464 -0.234355       3        1  10.516811
1     -0.021763  0.227010       4        1  10.276742
9      0.011896 -0.284057       5        1   9.187424
5     -0.012603  0.046268       6        1   7.674987
3      0.144834  0.107148       7        1   6.728817
7      0.151012 -0.039473       8        1   4.980575
6      0.091216 -0.003620       9        1   4.917884
0      0.263812  0.001685      10        1   2.588557, topic_info=            Term          Freq         Total Category  logprob  loglift
0             ai  69220.000000  69220.000000  Default  30.0000  30.0000
80    blockchain  25365.000000  25365.000000  Default  29.0000  29.0000
339       crypto  12866.000000  12866.000000  Default  28.0000  28.0000
525          nft  11616.000000  11616.000000  Default  27.0000  27.0000
244     business  16374.000000  16374.000000  Default  26.0000  26.0000
...          ...           ...           ...      ...      ...      ...
1458    critical    291.431645    429.143728  Topic10  -5.1309   3.2671
685         2022    241.451150    423.708593  Topic10  -5.3190   3.0917
262      founder    261.269036    690.962330  Topic10  -5.2401   2.6815
55          tech    366.929399  12476.060770  Topic10  -4.9005   0.1277
2482       daily    249.359149    995.358151  Topic10  -5.2868   2.2699

[546 rows x 6 columns], token_table=       Topic      Freq Term
term                       
163        1  0.550868   's
163        2  0.293582   's
163        3  0.098307   's
163        6  0.049020   's
163        8  0.003214   's
...      ...       ...  ...
874        2  0.970342    ”
874        7  0.029473    ”
4672       3  0.998247    •
5829       7  0.998538    ⁦
30066      7  0.998406    ✦

[922 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[9, 5, 3, 2, 10, 6, 4, 8, 7, 1])

---